In [1]:
import pandas as pd

In [ ]:
file = open('./data/aclImdb/train/pos/0_9.txt')

In [53]:
import os
import sys
import time
import re
import string
import unicodecsv as csv
import unicodedata as un
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

in_path = './data/aclImdb/test/'
out_path= './data/aclImdb/test.csv'



In [54]:
counter = 0
error_counter = 0

# Start timer
start_time = time.time()
label = 0

# Prepare dictionary of necessary unicode
# Thanks to https://stackoverflow.com/a/11066687/4595807
# We want to protect '-'
# HYPHEN-MINUS = UNICODE DECIMAL VALUE = 45
table = dict.fromkeys(i for i in range(sys.maxunicode) 
                        if un.category(chr(i)).startswith(('P','N','S','Cf','Cn','Cc')))

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

stop_words = set(stopwords.words('english'))

with open(out_path, 'wb') as out_file:
    writer = csv.writer(out_file, encoding='utf-8')
    
    for root, dirs, files in os.walk(in_path, topdown=True):
        
        for name in files:
            curr_file = os.path.join(root, name)
            try:
                # Read current file and remove BOM and newline characters
                # uf03c and uf03c are invalid unicode characters and don't have any category
                # so had to remove manually
                fp = open(curr_file, encoding='utf-8-sig').read()
                
                # Remove html tags
                fp = fp.replace('<br />', '')
                
                # Remove all punctuations
                fp = fp.translate(translator)
                
                # Remove numbers/symbols/invalid chars
                fp = fp.translate(table)
                
                # Conver to lower case
                fp = fp.lower()
                
                # Tokenize the word
                fp = word_tokenize(fp)
                
                # Remove the stopwords
                fp = [w for w in fp if not w in stop_words]
                
                
                # Normalize the unicode so that
                # canonical-equivalent ones will also have precisely the same binary representation
                final_msg = label, fp
                
                # Write into CSV file format - label, data
                writer.writerow(final_msg)

                # Counter setup to count file processed
                counter = counter + 1
#                 break

            except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
                error_counter = error_counter + 1
#                 break
        label += 1
                

out_file.close()
    
end_time = time.time()

print('Number of files processed: ',counter)
print('Number of files error: ',error_counter)

print('Time taken in seconds:',(end_time - start_time))

Number of files processed:  25000
Number of files error:  0
Time taken in seconds: 37.00022482872009
